## Documento para creación de ploteos para documento de conclusiones de ensayos dinámicos
* Autor: Gustavo Vladimir Diaz
* Fecha inicio: 5/3/2020
* Fecha última modificación: 5/3/2020

### Objetivo
Crear ploteos necesarios para entendimiento de conclusiones

#### Ploteos necesarios
* Conclusión "Adquisición con sensor óptico": Comparación entre ruido de ploteo de datos recogidos en sensor óptico y comparado con datos obtenidos en sensor inductivo
* Conclusión "Adquisición con osciloscopio": Mostrar que el tiempo de adquisición mientras mas largo es mayor resolución en frecuencia se va a tener, comparar uno que tenga adquisición de 6 segundos con alguno de 0.6 segundos
* Conclusión "Frecuencia fundamental": Mostrar algún ensayos cuya fundamental se observe plenamente
* Conclusión "Dispersión de fundamentales": Mostrar como por ejemplo en el ensayo 20 se observa como la fundamental tiene un corrimiento en frecuencia y no la energía concentrada en una frecuencia.
* Conclusión "Repetibilidad de ensayos": Mostrar como en el documento "Resultado 6" los distintos espectros como varían para un mismo rotor

In [ ]:
# Conclusión "Adquisición con sensor óptico":
